In [951]:
import nltk
import spacy
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np 
import re
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer

In [953]:
nlp= spacy.load("en_core_web_sm")

In [803]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)  #transcript stores in dictionaries inside list , where key is 'text'
        transcript_text = " ".join([item['text'] for item in transcript]) #to get values of all the dictionaries and combine them to form a string text.
        return transcript_text
    except Exception as e:
        print("Error fetching transcript:",e)
        return None


In [769]:
def clean_text(text):
   
    text = re.sub(r'\[.*?\]', '', text)  # Remove non- verbal sounds like [music]
    
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    
    text = text.strip()
    return text
    

In [771]:
def preprocess_text(text):
    text = clean_text(text)
    text = text.lower()
    doc=nlp(text)
    
    stemmer=PorterStemmer()
    stemmed_text = [ stemmer.stem(token.text) for token in doc ] #nltk

    lemmatized_text = [token.lemma_ for token in doc]  #spacy

    stop_words = set(stopwords.words('english'))
    filtered_text=[]
    
    for word in lemmatized_text:
        if word not in stop_words:
            filtered_text.append(word)

    text=" ".join(filtered_text)
    text = re.sub(r'[^\w\s]+', '', text)# Remove punctuation and music symbol or any symbol
    
    return text
    

In [971]:
def summarizer1(text, n=3):
    doc=nlp(text)
    
    processed_text = preprocess_text(text).split()
    
    pos_tags= pos_tag(processed_text) #stores as list of tuples 
    
    word_freq = {}
    for word,pos in pos_tags:
            if pos=="NN" or pos=="VBP":
                word_freq[word] = word_freq.get(word,0)+ 1 #default value is 0

    sentence_score={}
    for sentence in doc.sents:
        sentence_str = clean_text(sentence.text)
        for token in sentence:
            if token.text in word_freq:
                sentence_score[sentence_str] = sentence_score.get(sentence_str,0)+ word_freq[token.text]
                
    summarized_text= sorted(sentence_score, key=sentence_score.get, reverse=True )[:n]
    
    return ". ".join(s.capitalize() for s in summarized_text)

In [973]:
def summarizer2(text, n=3):
    doc = nlp(text)
    sentences= []
    for sentence in doc.sents:
        sentences.append(sentence.text)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    sentence_score = np.array(tfidf_matrix.sum(axis=1)).flatten()
    
    top_sent_indices= sentence_score.argsort()[-n:][::-1]
    
    top_sents= [sentences[i] for i in top_sent_indices]
    
    return ". ".join(s.capitalize() for s in top_sents)

In [975]:
video_url = input("Enter a youtube video url:")
n= int(input("Enter no. of lines of summary needed:"))

if "v=" in video_url:
    video_id = video_url.split("v=")[-1]
else:
    video_id = video_url.split("be/")[-1]     #returns a list
    
text = get_transcript(video_id)

print("\nSUMMARY OF THE TRANSCRIPT: OPTION 1")
summarized_text1= summarizer1(text, n)
print(summarized_text1)

print("\nSUMMARY OF THE TRANSCRIPT: OPTION 2")
summarized_text2= summarizer2(text, n)
print(summarized_text2)


Enter a youtube video url: https://www.youtube.com/watch?v=hOCDJyZ6quA
Enter no. of lines of summary needed: 3



SUMMARY OF THE TRANSCRIPT: OPTION 1
Okay. I'm going to link a good article here so in the sequential model you add layers one by one as a sequence you see but in a functional model you create your input then you create a hidden layer let's say and supply input as a function argument then you create hidden one then you supply that into hidden two's argument and so on and then you create model using inputs and outputs now this allows you to create a model which might have multiple inputs multiple outputs like something like rest net you know you can also share network layers with other other models so there are there are some differences you read the article you will get an idea so here i'm going to build a functional model. And when you use the about encoder it returns a dictionary out of which you need to use pulled out pull output is basically the encoding for the entire sentence again if you want to know what other elements are there in the dictionary you need to watch my previous v